In [2]:
'''
Random hint: if you want to add a package to a virtual environment, try the following:
USING CONDA


USING PIP
source activate ee 
pip install pygeoj #here, pip points to the pip in the ee virtual environment
'''

import ee
from IPython.display import Image

# Trigger the authentication flow.
ee.Authenticate()

In [3]:
ee.Initialize()

def get_images(path_list, row_list, satellite, start_date, end_date, max_cloud_percentage):
  coll = ee.ImageCollection(satellite).filterDate(start_date, end_date)\
            .filter(ee.Filter.inList('WRS_PATH', path_list))\
            .filter(ee.Filter.inList('WRS_ROW', row_list))\
            .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_percentage))
  image_ids = list(map(lambda x: x['id'], coll.getInfo()['features']))

  images = list(map(lambda x: ee.Image(x), image_ids))

  return images

In [4]:
p = [14]
r = [32]
sat = 'LANDSAT/LC08/C01/T1'
sd = '2013-05-01'
ed = '2020-05-01'
cc= 10

image_list_nyc = get_images(p, r, sat, sd, ed, cc)

image_list_nyc

In [4]:
!pwd

/home/arvindn/Research/TopicsOfInterest/Agriculture/Notes


In [5]:
'''
Functions needed to display images and features/featurecollections are given here. 
These functions are all from other sources. Mostly from google tutorials.
'''
import geojson
import json
import pygeoj
import numpy as np


import folium


# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [8]:
'''
In this cell, I show how you can take GeoJSON files and convert them into Google Earth Engine features and 
featurecollections. You might have created the GeoJSON files by some other means - one of them is
to use Google Earth to draw a polygon, save it as a kml file, and use ogr2ogr to conert  it to a GeoJSON file. I
created 4 such files, titled 'Hudson01.json', 'Hudson02.json', 'Bronx01.json', and 'Astoria01.json'. 

For those from NYC, or if you have a knowledge of the geography of NYC, you'd have guess  that 'Hudson01.json', and
'Hudson02.json' are regions in water, and that 'Bronx01.json', and 'Astoria01.json' correspond to land (not water).
Hence, I have created two lists jsonfiles and classalloc. landcover = 1 corresponds to water, and landcover = 0 to 
'not water'. 
'''

jsonfiles = ['Hudson01', 'Hudson02', 'Bronx01', 'Astoria01'];
classalloc = [1, 1, 0, 0];

'''
These dictionaries are to store intermediate objects before we get to features and feature collections. First,
we extract the data from the GeoJSON file. Then we extract the coordinates as a list. The list is converted into
a earth engine polygon. Finally, the polygon is converted into a feature. The feature has two attributes - 'name'
and 'class'. Both are taken from corresponding values of jsonfiles and classalloc.

Addendum: I realized, maybe wrongly, that the training data for supervized classification has to be given in the
form of points - discrete points (vector data), with each point having a property called 'landcover'. This 
landcover can take on values of 0 or 1. Hence, we need to convert the polygons we created (Hudson01.json, etc) 
into a collection of points. Whay I have done is to take the polygonal regions and determine approximately 1000
random points within the region as follows: 

ee.FeatureCollection.randomPoints(region=ee_dict[jsonfile + 'ee'],points=1000)

The above function picks random points within the area given by the keyword region. Now, this function creates a 
featurecollection of random points within a polygonal region but does not assign a property called landcover. That
is done by the next few lines. 

def addProp(feature):
    return feature.set({'landcover': classalloc[n]})

randomPoints = randomPoints.map(addProp) #This is to add a property named

The function addProp takes and feature and adds a property called landcover, the value of which is from a value
outside the scope of the function. Not sure if this is good practice, but it works. The next line with map 
automatically applies it to every feature in a feature collection. No need for messy for loops.

We need to merge all these featurecollections into one final featurecollection that can be used for training.
That is what is achieved by the lines:

fc = ee.FeatureCollection([])
for x in randomPts_dict.keys():
    print(x)
    fc = fc.merge(randomPts_dict[x])

    
'''
coords_dict = {}
ee_dict = {}
randomPts_dict = {}
features_dict = {}

n = 0
for jsonfile in jsonfiles:
    jsonfilepath = '/home/arvindn/Research/TopicsOfInterest/Agriculture/Notes/'+ jsonfile +'.json'
    with open(jsonfilepath) as f:
        data = geojson.load(f)
    
    #creating a dictionary of coordinates
    coords_dict[jsonfile + 'coords'] = np.array(data['features'][0]['geometry']['coordinates'][0])[:,0:2].tolist()
    
    #creating a polygon from coordinate list
    ee_dict[jsonfile + 'ee'] = ee.Geometry.Polygon(coords_dict[jsonfile + 'coords'])
    
    
    #randomPoints = ee.FeatureCollection.randomPoints(region=ee_dict[jsonfile + 'ee'],points=100, {'name': jsonfile, 'landcover': classalloc[n]})
    randomPoints = ee.FeatureCollection.randomPoints(region=ee_dict[jsonfile + 'ee'],points=30)
    
    def addProp(feature):
      return feature.set({'landcover': classalloc[n]})

    randomPoints = randomPoints.map(addProp) #This is to add a property named
    
    #randomPts_dict[jsonfile+'Pts'] = ee.Feature(ee.Geometry.MultiPoint(ee.List(coords_dict[jsonfile + 'coords'])), 
    #                                            {'name': jsonfile, 'class': classalloc[n]})
    
    randomPts_dict[jsonfile+'Pts'] = randomPoints
    
    
    
    #randomPts_dict[jsonfile + 'rdnmPts'] = ee.FeatureCollection.randomPoints(ee_dict[jsonfile + 'ee'], 100)
    features_dict[jsonfile + 'feature'] = ee.Feature(ee_dict[jsonfile + 'ee'], {'name': jsonfile, 'landcover': classalloc[n]})

    n = n+1

    
'''
The individual features are combined as shown below to create a feature collection.

You can get some information about the features in the collection using commands as shown below.
'''
nycFC = ee.FeatureCollection(list(features_dict.values()))

nycFCpts = ee.FeatureCollection(list(randomPts_dict.values()))

fc = ee.FeatureCollection([])
for x in randomPts_dict.keys():
    #print(x) #I had used this to make sure the loop was okay
    fc = fc.merge(randomPts_dict[x])


print(nycFC.size().getInfo())

print(type(nycFC.getInfo()))

print(nycFC.getInfo()['features'][3]['properties'])


'''
Displaying a featurecollection
'''
nycFC_centroid = nycFC.geometry().centroid().getInfo()['coordinates']
nycFC_centroid.reverse()
#Mapdisplay(nycFC_centroid,{'nycFC':nycFC.getInfo()},zoom_start=10)
Mapdisplay(nycFC_centroid, {'rndmPts': fc.getInfo()}, zoom_start=10)

4
<class 'dict'>
{'landcover': 0, 'name': 'Astoria01'}


In [7]:
#Various commands to interrogate the nature of nycFC or any featurecollection
#print(nycFC.getInfo()['features'][3]) 
print(nycFC.getInfo()['type'])

FeatureCollection


In [8]:
#To determine number of eatures in a featurecollection
nycFC.size().getInfo()

4

In [282]:
print(nycFC.geometry().getInfo()['type']) #multpolygon
print(fc.geometry().getInfo()['type']) #Multipoint

fc.geometry().geometries().getInfo()
#Try nycFC.geometry().geometries().getInfo() to see the difference between nycFC and fc

MultiPolygon
MultiPoint


[{'type': 'Point', 'coordinates': [-73.97726913448903, 40.80338152219788]},
 {'type': 'Point', 'coordinates': [-73.96308775134366, 40.82843265930131]},
 {'type': 'Point', 'coordinates': [-73.95761005004844, 40.84776534504073]},
 {'type': 'Point', 'coordinates': [-73.98212436310615, 40.79733458724368]},
 {'type': 'Point', 'coordinates': [-73.95515245391664, 40.835706162212276]},
 {'type': 'Point', 'coordinates': [-73.98454721702588, 40.796822194775984]},
 {'type': 'Point', 'coordinates': [-73.98541469961975, 40.79179870176999]},
 {'type': 'Point', 'coordinates': [-73.9617751516699, 40.83534902988346]},
 {'type': 'Point', 'coordinates': [-73.9864166821032, 40.79159075870921]},
 {'type': 'Point', 'coordinates': [-73.98286424708584, 40.79560622044268]},
 {'type': 'Point', 'coordinates': [-73.97524317791985, 40.81207140261596]},
 {'type': 'Point', 'coordinates': [-73.9528559673429, 40.84238548075046]},
 {'type': 'Point', 'coordinates': [-73.9727898136525, 40.818797535565636]},
 {'type': 'Po

In [9]:
# What properties do the constituent features of the featureCollection have?
#nycFC.getInfo()['features'][0]['properties']
fc.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.97726913448903, 40.80338152219788]},
   'id': '1_1_1_2_0',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.96308775134366, 40.82843265930131]},
   'id': '1_1_1_2_1',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.95761005004844, 40.84776534504073]},
   'id': '1_1_1_2_2',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.98212436310615, 40.79733458724368]},
   'id': '1_1_1_2_3',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.95515245391664, 40.835706162212276]},
   'id': '1_1_1_2_4',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates

In [8]:
#How to join features into a featurecollection
'''
This is code I took from 
https://colab.research.google.com/github/csaybar/EEwPython/blob/master/4_features.ipynb#scrollTo=G-JXF_byXB01
to understand how to combine features into featurecollections.
'''
features_list_example = [
  ee.Feature(ee.Geometry.Rectangle(30.01, 59.80, 30.59, 60.15), {'name': 'Voronoi'}),
  ee.Feature(ee.Geometry.Point(-73.96, 40.781), {'name': 'Thiessen'}),
  ee.Feature(ee.Geometry.Point(6.4806, 50.8012), {'name': 'Dirichlet'})
];

fromList = ee.FeatureCollection(features_list_example) 

print(fromList.size().getInfo())

3


In [9]:
nycsite = ee.Geometry.Rectangle([-74.04, 40.69, -73.82, 40.94]);

image0 = image_list_nyc[0] #one image from the list of images

parameters = {'min': 0.0,
              'max': 16000.0,
              'dimensions': 600,
              'bands': ['B4', 'B3', 'B2']}

Image(url = image0.clip(nycsite).getThumbUrl(parameters))

In [10]:
'''
Second step in any supervised classification process - training using the training data. The corresponding
code in Google EE tutorial is as follows:
// Get the values for all pixels in each polygon in the training.
var training = image.sampleRegions({
  // Get the sample from the polygons FeatureCollection.
  collection: polygons,
  // Keep this list of properties from the polygons.
  properties: ['class'],
  // Set the scale to get Landsat pixels in the polygons.
  scale: 30
});

The code is in javascript. I had to convert it to Python. Mistakes could be created/transmitted because of this
translation
'''
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']
#bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

training = image0.select(bands).sampleRegions(collection =  fc, properties = ['landcover'], scale = 30.0)
#print(training.geometry().getInfo()['type'])

'''
We can view feature collections using Mapdisplay, the function we encountered earlier. THe way to do it is as follows:
Mapdisplay(center,dicc={'poly1':hudson02ee.getInfo(),'poly2':hudson01ee.getInfo(), 'poly3': Astoria01ee.getInfo(), 'poly4': Bronx01ee.getInfo()},zoom_start=2)
'''

training_centroid = training.geometry().centroid().getInfo()['coordinates']
training_centroid.reverse()

'''
training_centroid is also a featurecollection but different from nycFC. Try print(training.size().getInfo()) - 
you'll see that the size is about 20000, not 4. Since I cannot display training, I think there must be a mistake 
right here. Need to rectify this - understand image.sampleRegions better before proceeding further.
'''

"\ntraining_centroid is also a featurecollection but different from nycFC. Try print(training.size().getInfo()) - \nyou'll see that the size is about 20000, not 4. Since I cannot display training, I think there must be a mistake \nright here. Need to rectify this - understand image.sampleRegions better before proceeding further.\n"

In [350]:
'''
defining the classifier - is that what this step is? 
The corresponding code in Google EE tutorial from which I generated this is as follows
// Create an SVM classifier with custom parameters.
var classifier = ee.Classifier.libsvm({
  kernelType: 'RBF',
  gamma: 0.5,
  cost: 10
});

'''
#classifier = ee.Classifier.libsvm(kernelType = 'RBF', gamma = 0.5, cost = 10)
'''
Turns out the choice of the classifier is important. The choice in this cell 
ee.Classifier.libsvm(kernelType = 'RBF', gamma = 0.5, cost = 10)
did not seem to work out well. The ee.Classifier.smileCart() worked wonders. Really surprised. 
Speaks volumes about how much of a blackbox there is in machine learning.
'''

In [13]:
training.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'properties': {'band_order': ['B2',
   'B3',
   'B4',
   'B5',
   'B6',
   'B7',
   'B10',
   'B11']},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_0_0',
   'properties': {'B10': 25821,
    'B11': 23754,
    'B2': 11031,
    'B3': 10460,
    'B4': 9760,
    'B5': 7982,
    'B6': 7081,
    'B7': 6632,
    'landcover': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_1_0',
   'properties': {'B10': 25826,
    'B11': 23761,
    'B2': 11252,
    'B3': 10765,
    'B4': 10116,
    'B5': 8386,
    'B6': 7471,
    'B7': 6932,
    'landcover': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_2_0',
   'properties': {'B10': 25910,
    'B11': 23806,
    'B2': 10960,
    'B3': 10415,
    'B4': 9754,
    'B5': 7992,
    'B6': 6992,
    'B7': 6597,
    'landcover': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_3_0',
   'properties': {'B10': 25814,
    'B11': 23783,
    'B2':

In [11]:
#Train the classifier.
'''
// Train the classifier.
var trained = classifier.train(training, 'class', bands);
'''
#trained = classifier.train(training, 'landcover', bands);
trained = ee.Classifier.smileCart().train(training, 'landcover', bands)
print(type(trained))

<class 'ee.Classifier'>


In [12]:
#Classify the image.
'''
This is the step at which the code is failing. If I try to interrogate classified in anyway, the computation is 
never completed. For example, if I did classified.getInfo(), an asterix * would appear next to the cell, never to 
finish.
'''
classified = image0.select(bands).classify(trained);

print(type(classified))

<class 'ee.image.Image'>


In [16]:
type(classified)

ee.image.Image

In [14]:
# Create a folium map object.
my_map = folium.Map(location=[ 40.8, -73.8], zoom_start=9, height=500)
#location should be given as latitude, longitude

my_map.add_ee_layer(image0, {'bands': ['B4', 'B3', 'B2'], 'max': 16000, 'gamma': 2}, 'image')

display(my_map)

In [15]:
my_map.add_ee_layer(classified, {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'water')
display(my_map)

In [16]:
from IPython.display import Image
parameters = {'min': 0.0,
              'max': 1.5,
              'dimensions': 400,
              'palette': ['black', 'white'],
              #'region': ee.Geometry.Rectangle([-74.04, 40.69, -73.82, 40.94])
}
Image(url = classified.getThumbUrl(parameters))




In [346]:
from ee import batch

## Make 8 bit data
def convertBit(image):
    return image.multiply(512).uint8()  

outputImage = classified.multiply(512).uint8()  

out = batch.Export.image.toDrive(image = outputImage, description = 'imageToDriveExample',  scale = 30);

process = batch.Task.start(out)

print("Process sent to cloud")

Process sent to cloud


In [ ]:
buffer = hudson02ee.buffer(100).getInfo()

# Compute the centroid of the polygon.
centroid = hudson02ee.centroid().getInfo()


#Go Folium
center = centroid['coordinates']

Mapdisplay(center,{'center':centroid,'buffer':buffer},zoom_start=3)

In [174]:
Mapdisplay(center,dicc={'poly1':hudson02ee.getInfo(),'poly2':hudson01ee.getInfo(), 'poly3': Astoria01ee.getInfo(), 'poly4': Bronx01ee.getInfo()},zoom_start=2)

In [9]:
jsonfiles
jsonfilepath

['/home/arvindn/Research/TopicsOfInterest/Agriculture/Notes/Hudson01.json',
 '/home/arvindn/Research/TopicsOfInterest/Agriculture/Notes/Hudson02.json',
 '/home/arvindn/Research/TopicsOfInterest/Agriculture/Notes/Bronx01.json',
 '/home/arvindn/Research/TopicsOfInterest/Agriculture/Notes/Astoria01.json']

In [53]:

fromList

In [54]:
nycFC_centroid = nycFC.geometry().centroid().getInfo()['coordinates']
fromList_centroid = fromList.geometry().centroid().getInfo()['coordinates']

In [55]:
fromList_centroid

[30.300000000000317, 59.975009662334436]

In [62]:
nycFC.size().getInfo()

4

In [339]:
training.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'properties': {'band_order': ['B2',
   'B3',
   'B4',
   'B5',
   'B6',
   'B7',
   'B10',
   'B11']},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_0_0',
   'properties': {'B10': 25821,
    'B11': 23754,
    'B2': 11031,
    'B3': 10460,
    'B4': 9760,
    'B5': 7982,
    'B6': 7081,
    'B7': 6632,
    'landcover': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_1_0',
   'properties': {'B10': 25826,
    'B11': 23761,
    'B2': 11252,
    'B3': 10765,
    'B4': 10116,
    'B5': 8386,
    'B6': 7471,
    'B7': 6932,
    'landcover': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_2_0',
   'properties': {'B10': 25910,
    'B11': 23806,
    'B2': 10960,
    'B3': 10415,
    'B4': 9754,
    'B5': 7992,
    'B6': 6992,
    'B7': 6597,
    'landcover': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_3_0',
   'properties': {'B10': 25814,
    'B11': 23783,
    'B2':

In [93]:
type(training)

ee.featurecollection.FeatureCollection

In [345]:
print(training.size().getInfo())

120


In [42]:
training.style().getInfo()

{'type': 'Image',
 'bands': [{'id': 'vis-red',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'vis-green',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'vis-blue',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [57]:
training.geometry().getInfo()

{'geodesic': False, 'type': 'MultiPoint', 'coordinates': []}

In [61]:
nycFC.geometry().getInfo()['type']

'MultiPolygon'

In [62]:
training.geometry().getInfo()['type']

'MultiPoint'

In [94]:
type(nycFC.geometry())

ee.geometry.Geometry

In [137]:
nycFCrandom = ee.FeatureCollection.randomPoints(nycFC.geometry(), 20)

In [138]:
type(nycFCrandom)

ee.featurecollection.FeatureCollection

In [103]:
nycFCrandom02 = nycFCrandom.copyProperties(features_dict['Hudson01' + 'feature'], ['class'])

In [99]:
type(features_dict['Hudson01' + 'feature'])

ee.feature.Feature

In [110]:
def copyProps(feature, feat2copyFrom):
  return feature.copyProperties(feat2copyFrom, ['class'])

#nycFCrandom02  = nycFCrandom.map(copyProps, features_dict['Hudson01' + 'feature'])

In [111]:
points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

In [143]:
nycFrandomPts03 = ee.Feature(ee.Geometry.MultiPoint(nycFCrandom.geometry().geometries()), {'class': 0})
print(type(nycFCrandom.geometry().geometries()))
print(type(ee.List(coords_dict[jsonfile + 'coords'] )))

<class 'ee.ee_list.List'>
<class 'ee.ee_list.List'>


In [140]:
nycFrandomPts03.getInfo()
#type(nycFCrandom.geometry().geometries())

{'type': 'Feature',
 'geometry': {'type': 'MultiPoint',
  'coordinates': [[-73.89658322122182, 40.81844704596529],
   [-73.92207072978285, 40.81823305690301],
   [-73.93790437373124, 40.86650060935002],
   [-73.88872287665208, 40.812199393651085],
   [-73.96887952217843, 40.81401989067663],
   [-73.92407807890261, 40.76248766025991],
   [-73.93515390627881, 40.88971716960757],
   [-73.91285130235313, 40.80269561504147],
   [-73.91447385310862, 40.81534276521771],
   [-73.91209369828665, 40.810669836187955],
   [-73.91960538124022, 40.765542769695834],
   [-73.96254525447385, 40.82405733856507],
   [-73.99236070870475, 40.78899521182378],
   [-73.92083015014255, 40.809999490260346],
   [-73.91611783128813, 40.83017167599397],
   [-73.88864549884244, 40.81979710185069],
   [-73.9101673327282, 40.8213062659063],
   [-73.91025711020956, 40.82754202611094],
   [-73.9342164093891, 40.89042442756619],
   [-73.92468043769962, 40.76702223189583]]},
 'properties': {'class': 0}}

In [145]:
nycFCpts.getInfo()

{'type': 'FeatureCollection',
 'columns': {'class': 'Integer', 'name': 'String', 'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPoint',
    'coordinates': [[-73.957327, 40.849796],
     [-73.957484, 40.849313],
     [-73.95764, 40.84883],
     [-73.957796, 40.848348],
     [-73.957991, 40.847744],
     [-73.958148, 40.847261],
     [-73.958343, 40.846658],
     [-73.958659, 40.846205],
     [-73.958854, 40.845601],
     [-73.95901, 40.845119],
     [-73.959326, 40.844666],
     [-73.959483, 40.844183],
     [-73.959678, 40.843579],
     [-73.959873, 40.842976],
     [-73.96003, 40.842493],
     [-73.960186, 40.84201],
     [-73.960502, 40.841557],
     [-73.960658, 40.841074],
     [-73.960814, 40.840592],
     [-73.96113, 40.840138],
     [-73.961286, 40.839656],
     [-73.961482, 40.839052],
     [-73.961638, 40.838569],
     [-73.961794, 40.838087],
     [-73.961989, 40.837483],
     [-73.962224, 40.836759],
     [-73.962419, 40.836156],
 

In [148]:
randomPoints.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.92203629357051, 40.763676621925164]},
   'id': '0',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.91871237083838, 40.76422849735108]},
   'id': '1',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.92881354132867, 40.76479098474038]},
   'id': '2',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.92132891671262, 40.765923751402326]},
   'id': '3',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.93023172339797, 40.76672229998491]},
   'id': '4',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.92427437208441, 40.76400309696836]},
   'id': '5',
   'properties': {}},
  {'type': 'Feature',
   'ge

In [188]:
randomPoints.getInfo()

{'type': 'FeatureCollection',
 'columns': {'landcover': 'Integer'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.92203629357051, 40.763676621925164]},
   'id': '0',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.91871237083838, 40.76422849735108]},
   'id': '1',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.92881354132867, 40.76479098474038]},
   'id': '2',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.92132891671262, 40.765923751402326]},
   'id': '3',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.93023172339797, 40.76672229998491]},
   'id': '4',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.924274372

In [187]:
def addProp(feature):
  return feature.set({'landcover': 1})

randomPoints = randomPoints.map(addProp)

In [182]:
ee.Feature(ee.Geometry.MultiPoint(randomPoints.geometry().geometries()), {'landcover':0}).getInfo()

{'type': 'Feature',
 'geometry': {'type': 'MultiPoint',
  'coordinates': [[-73.92203629357051, 40.763676621925164],
   [-73.91871237083838, 40.76422849735108],
   [-73.92881354132867, 40.76479098474038],
   [-73.92132891671262, 40.765923751402326],
   [-73.93023172339797, 40.76672229998491],
   [-73.92427437208441, 40.76400309696836],
   [-73.91749214913304, 40.76251164401785],
   [-73.91757137161306, 40.762259444594115],
   [-73.92589614540735, 40.76595893833707],
   [-73.9206436520613, 40.7661418915763],
   [-73.9263560939088, 40.764577921904696],
   [-73.91497323358605, 40.76594765279363],
   [-73.9155900370266, 40.76623697331911],
   [-73.9280798169411, 40.76488190305168],
   [-73.91939868959375, 40.76301930976759],
   [-73.91929357605814, 40.767526024838816],
   [-73.92410845571196, 40.7670701260946],
   [-73.92330899999226, 40.76808235624977],
   [-73.92823900913149, 40.765432756821625],
   [-73.92255672843007, 40.76609871918763],
   [-73.92615193718883, 40.76465436186263],
   [-

In [172]:
features_dict['Hudson01feature'].propertyNames().getInfo()

['class', 'name']

In [209]:
randomPts_dict['Hudson02Pts'].getInfo()

{'type': 'FeatureCollection',
 'columns': {'class': 'Integer'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.93552247679578, 40.872233618646526]},
   'id': '0',
   'properties': {'class': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.94363938214886, 40.86232822342618]},
   'id': '1',
   'properties': {'class': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.94798588359137, 40.85448597612985]},
   'id': '2',
   'properties': {'class': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.93621587788225, 40.88706891403025]},
   'id': '3',
   'properties': {'class': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.93513369957786, 40.86925833863237]},
   'id': '4',
   'properties': {'class': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.93616556543076, 40.88248108674617]

In [211]:
xyz = randomPts_dict['Hudson01Pts'].merge(randomPts_dict['Hudson02Pts'])

In [213]:
xyz = xyz.merge(randomPts_dict['Astoria01Pts'])

In [214]:
xyz.getInfo()

{'type': 'FeatureCollection',
 'columns': {'class': 'Integer'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.97726913448903, 40.80338152219788]},
   'id': '1_1_0',
   'properties': {'class': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.96308775134366, 40.82843265930131]},
   'id': '1_1_1',
   'properties': {'class': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.95761005004844, 40.84776534504073]},
   'id': '1_1_2',
   'properties': {'class': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.98212436310615, 40.79733458724368]},
   'id': '1_1_3',
   'properties': {'class': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.95515245391664, 40.835706162212276]},
   'id': '1_1_4',
   'properties': {'class': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.98454721702588

In [220]:
randomPts_dict.keys()

dict_keys(['Hudson01Pts', 'Hudson02Pts', 'Bronx01Pts', 'Astoria01Pts'])

In [231]:
fc = ee.FeatureCollection([])
for x in randomPts_dict.keys():
    print(x)
    fc = fc.merge(randomPts_dict[x])

Hudson01Pts
Hudson02Pts
Bronx01Pts
Astoria01Pts


In [247]:
training.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'properties': {'band_order': ['B2',
   'B3',
   'B4',
   'B5',
   'B6',
   'B7',
   'B10',
   'B11']},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_0_0',
   'properties': {'B10': 25821,
    'B11': 23754,
    'B2': 11031,
    'B3': 10460,
    'B4': 9760,
    'B5': 7982,
    'B6': 7081,
    'B7': 6632,
    'landcover': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_1_0',
   'properties': {'B10': 25826,
    'B11': 23761,
    'B2': 11252,
    'B3': 10765,
    'B4': 10116,
    'B5': 8386,
    'B6': 7471,
    'B7': 6932,
    'landcover': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_2_0',
   'properties': {'B10': 25910,
    'B11': 23806,
    'B2': 10960,
    'B3': 10415,
    'B4': 9754,
    'B5': 7992,
    'B6': 6992,
    'B7': 6597,
    'landcover': 1}},
  {'type': 'Feature',
   'geometry': None,
   'id': '1_1_1_2_3_0',
   'properties': {'B10': 25814,
    'B11': 23783,
    'B2':

In [267]:
fc.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.97726913448903, 40.80338152219788]},
   'id': '1_1_1_2_0',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.96308775134366, 40.82843265930131]},
   'id': '1_1_1_2_1',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.95761005004844, 40.84776534504073]},
   'id': '1_1_1_2_2',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.98212436310615, 40.79733458724368]},
   'id': '1_1_1_2_3',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-73.95515245391664, 40.835706162212276]},
   'id': '1_1_1_2_4',
   'properties': {'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates